# $orvara$ tutorial

### cd into the root directory and install the package first with pip install

This will build orvara. Make sure that you are in the root directory of the repo. The -e flag builds the Cython modules.

In [1]:
ls

CHANGES.md                Tutorial.ipynb            orvara.egg-info/
HGCA_vDR2_corrected.fits  build/                    pyproject.toml
LICENCE                   misc/                     pytest.ini
README.rst                orvara/                   setup.py


In [ ]:
pip install -e .

### Run pytest to make sure that the tests pass

This will run a small suite of tests. This should take about 5 minutes. If any of the tests fail, something is wrong with the install. Try running pip install -e . If the issue persists, please submit an issue ticket!

In [2]:
!pytest -sv

============================= test session starts ==============================
platform darwin -- Python 3.8.5, pytest-6.1.1, py-1.9.0, pluggy-0.13.1 -- /Users/user/anaconda3/bin/python
cachedir: .pytest_cache
rootdir: /Users/user/orvara, configfile: pytest.ini
collected 5 items                                                              

orvara/tests/test_config.py::test_example_config_file PASSED
orvara/tests/test_main.py::test_set_initial_parameters PASSED
orvara/tests/test_main.py::test_get_priors PASSED
orvara/tests/test_main.py::test_run Loading RV data from file orvara/tests/data/Gl758_RV.dat
Loaded data from 3 RV instruments.
Loading astrometric data for 1 planets
Loaded 4 relative astrometric data points from file orvara/tests/data/Gl758_relAST.txt
Loading absolute astrometry data for Hip 95319
Not using companion proper motion from Gaia.
Using ptemcee.
Running MCMC ... 
[###############################]100%50%
Total Time: 2 seconds
Mean acceptance fraction (cold chain): 0

### Check the usage of fit_orbit and plot_orbit

fit_orbit and plot_orbit are the key commands to operate this package. Run --h to see their functionality. 

both commands require one optional argument --output-dir immediately followed by the desired directory to write output to. fit_orbit outputs the MCMC chain from the fit and plot_orbit outputs a set of publication-quality plots. If --output-dir flag is omitted, the output directory will be automatically set to the current working directory by default. 

Following the output directory is the configuration file tailored to the system you are fitting. Example configuration files can be found in /orvara/tests/.

In [3]:
!fit_orbit --h

usage: fit_orbit [-h] [--output-dir OUTPUT_DIR] configfile

Fit a Keplerian orbit or orbits using orvara.

positional arguments:
  configfile            Path to the planet-specific configuration file.

optional arguments:
  -h, --help            show this help message and exit
  --output-dir OUTPUT_DIR
                        Directory within which to save the MCMC results,
                        default current directory.


In [4]:
!plot_orbit --h

usage: plot_orbit [-h] [--output-dir OUTPUT_DIR] configfile

Plot astrometry or RV orbits and other relavant plots.

positional arguments:
  configfile            Path to the planet-specific configuration file.

optional arguments:
  -h, --help            show this help message and exit
  --output-dir OUTPUT_DIR
                        Directory within which to save the plots, default
                        current directory.


### Running an example of fit_orbit, we will fit orbit to the White Dwarf - Main Sequence (WD-MS) binary system HD159062. 

The customizable configuration file for HD 159062 is located in orvara/tests/

So let's load the config.ini file for HD 159062 here:

In [5]:
ls orvara/tests/

__pycache__/                      data/
chains/                           diagnostic_config.ini
config.ini                        plots/
config_Gl758.ini                  start_Gl758.dat
config_HD159062.ini               test_config.py
config_HD4747.ini                 test_main.py
config_with_secondary_priors.ini  utils.py


In [ ]:
# %load orvara/tests/config_HD159062.ini
[data_paths]
# Hipparcos ID of the star, used for fetching absolute astrometry.
# If the Hipparcos ID is invalid the fit will not use HGCA astrometry. 
# In that case you must supply a parallax and parallax_error in [priors].
HipID = 85653
# The Hipparcos Gaia Catalog
HGCAFile = HGCA_vEDR3_withchi2.fits
# The file containing the radial velocity time series for the star.
RVFile = orvara/tests/data/HD159062_RV.dat
# The file containing the relative astrometry for the star. 
AstrometryFile = orvara/tests/data/HD159062_relAST.txt
# The path to all the Gaia DR2 intermediate data
GaiaDataDir = orvara/tests/data/gaia
# The path to all the Hipparcos (original reduction) intermediate data
Hip1DataDir = orvara/tests/data/hip1
# The path to all the Hipparcos (second reduction) intermediate data. Note that this should be from the Java Tool. See the orvara readme for 
# a description and a link to download.
Hip2DataDir = orvara/tests/data/hip2
# the file path to the initial conditions to the orbit. 
# Set to None for default guess.
start_file = None

[mcmc_settings]
# number of temperatures to use in the parallel tempering chain
ntemps = 10
# number of walkers. Each walker will have ntemps number of chains.
nwalkers = 100
# number of planets to fit.
nplanets = 1
# number of steps contained in each chain
nstep = 20000
# How much to thin the chain (save every thin-th step)
thin = 50
# number of threads to use with emcee. Note this built-in parellelization is poor.
nthreads = 1
# True if you want to use the epoch astrometry in GaiaDataDir, Hip1DataDir etc... False if not.
use_epoch_astrometry = True
# Use a separate jitter for each RV instrument?
jit_per_inst = False

[priors_settings]
# priors on primary mass (solar), if prior is not specified, mpri should be inf
mpri = 0.8
mpri_sig = 0.05
minjitter = 1e-5
maxjitter = 1e3

[secondary_gaia]
# If the secondary star is in Gaia, set companion_ID to a nonnegative number
# matching the ID of the companion in the relative astrometry file.  Setting
# companion_ID to a negative number ignores the rest of this.
companion_ID = -1
# The rest of these should be from Gaia in units of mas.
pmra = 0
pmdec = 0
epmra = 100
epmdec = 100
corr_pmra_pmdec = 0

[plotting]
# Path to mcmc chains. This is what's produced by the orbit_fit command.
McmcDataFile = orvara/tests/chains/HD159062B_chain000.fits
# Diagnostic plots to check for convergence.
check_convergence = True
# Define burnin for chains.
burnin = 50
# Which companion to plot?
iplanet = 0

# Name of the target
target = HD159062B
# This is a customized range of epochs you want to plot.
start_epoch = 1990.
end_epoch = 2015.
# Number of random orbits drawn from the posterior distribution you want to plot.
num_orbits = 50
# Define step size for plotting.
num_steps = 1500
# Plot random predicted epoch positions on the Astrometry plot.
predicted_years = 1990,2000,2010,2020,2030
# Prediction of the location of the companion in a future epoch.
position_predict = 2010


# Select which plot you want to generate
Astrometry_orbits_plot = True
Astrometric_prediction_plot = True
RV_orbits_plot = True
RV_plot = True
RV_Instrument = All
Relative_separation_plot = True
Position_angle_plot = True
Proper_motion_plot = True
Proper_motion_separate_plots = False
Corner_plot = True

############# Advanced plotting settings #############
# 1. Set axes limits. Set the upper and lower limit for x and y axes.
set_limit = False
xlim = 1980, 2025
ylim = -2.8,2.8
# Choose the color of the marker for plotting the observed data points.
marker_color = blue

# 2. Turn on/off colorbar. Choose a matplotlib colormap, and choose to color
# code by the secondary mass (msec_solar or msec_jup) or eccentricity (ecc).
use_colorbar = True
colormap = viridis
reference = msec_solar

# 3. Turn on/off the title of the plot? Additionally, if user wants to add
# text somewhere on the plot, enter the text and its x and y position (with
# (0, 0) being the bottom left and (1, 1) being the top right).
show_title = False
add_text = True
text_name = HD 159062B
x_text = 0.5
y_text = 0.9


[save_results]
# quantiles: median and 1sigma = 0.16, 0.5, 0.84, 
# median and 2sigma = 0.025, 0.5, 0.975, or arbitrary
save_params = True
err_margin = 0.16, 0.5, 0.84

Next, let's load the data sets needed for orbit fitting. There are three data types in general: RV, relative astrometry, or/and absolute astrometry. HD 159062 has all. Example data files are located in orvara/tests/data. Let's display them here:

In [8]:
ls orvara/tests/data

Gl758_RV.dat         HD159062_relAST.txt  gaia/
Gl758_relAST.txt     HD4747_RV.dat        hip1/
HD159062_RV.dat      HD4747_relAST.txt    hip2/


This is the RV data, it has four columns: Epoch[JD], RV[m/s], RV_err[m/s] and InstrumentID

In [ ]:
# %load orvara/tests/data/HD159062_RV.dat
# Instrument: HIRES on Keck
# Published in Hirsch, L. A., Ciardi, D. R., Howard, A. W., et al. 2019, ApJ, 878, 50
# File format:
# Epoch[JD]   RV[m/s] RV_err[m/s] InstrumentID
2452832.91668	89.22	1.62	0
2453074.09155	83.23	1.68	0
2453196.81631	76.88	1.58	0
2453430.15779	65.68	0.93	0
2453547.94513	60.98	0.84	0
2453604.84622	57.70	1.03	0
2453807.13748	50.30	1.00	0
2453926.45063	43.33	0.57	0
2453927.88530	44.96	0.83	0
2454248.03230	31.71	1.44	0
2454249.95458	32.00	1.49	0
2454252.04380	32.81	1.02	0
2454255.93510	32.28	1.01	0
2454277.86337	31.99	1.33	0
2454278.90748	32.22	1.22	0
2454279.94547	31.42	1.29	0
2454285.90800	30.58	1.34	0
2454294.90713	30.60	1.30	0
2454634.43562	19.74	1.03	0
2454635.94255	19.00	1.43	0
2454636.93316	17.04	1.53	0
2454638.33247	18.76	0.99	0
2454640.38633	14.57	1.06	0
2454641.94950	19.55	1.47	0
2454644.08325	15.11	1.52	0
2454688.88580	17.38	1.47	0
2454689.89920	9.37	1.58	0
2454957.13845	3.77	1.67	0
2455401.78282	-16.12	1.40	0
2455722.86450	-22.42	1.56	0
2455782.89029	-26.95	1.55	0
2456164.72278	-40.66	1.54	0
2456451.97582	-50.77	1.03	0
2456475.78721	-49.97	1.43	0
2456709.08945	-54.30	1.43	0
2456883.74892	-65.35	1.49	0
2456910.83550	-66.37	1.62	0
2457061.17682	-66.74	1.43	0
2457211.95513	-76.17	1.40	0
2457671.70374	-91.12	1.72	0
2457831.16397	-94.97	1.59	0
2457853.14530	-91.51	1.92	0
2457970.94501	-103.25	1.86	0
2458263.03029	-108.99	1.22	0
2458349.72648	-110.69	1.58	0

This is the relative astrometry file for HD 159062. It has seven columns: Date      , Sep (arcsec) , Err_sep  ,    PA (degrees),  Err_PA  Corr_Sep_PA, and  PlanetID. Planet ID depends on how many planets are in the system, 0 denotes the innermost planet, the second innermost is denoted 1 and so on.

In [ ]:
# %load orvara/tests/data/HD159062_relAST.txt
# Instrument: NIRC2 on Keck
# Published in Hirsch, L. A., Ciardi, D. R., Howard, A. W., et al. 2019, ApJ, 878, 50
##
# File format
#   Date       Sep  Err_sep      PA  Err_PA  Corr_Sep_PA  PlanetID  
 2012.48     2.594    0.014   298.6     0.1            0         0
 2014.78     2.637    0.014   299.5     0.1            0         0
 2018.32     2.673    0.003  301.27    0.04            0         0

For absolute astrometry from the HGCA, make sure that the HGCA file (DR2 or eDR3) is specified. We are using Gaia eDR3 version of the HGCA.

To use intermediate astrometry, make sure that the parameter use_epoch_astrometry in the config file is set to True. Also, make sure that the Gaia and Hipparcos intermediate data are ready to be fetched by htof. The Gaia eDR3 observations are publicly accessible via the Gaia Observation Forecast Tool (https : //gaia.esac.esa.int/gost/index.jsp). Gaia files are .csv files queried from GOST. The Hip1 and Hip2 data are (if not, soon) avaliable publically from ESA.

Here, names containing HIP85653 are the ones we want for HD 159062.

In [11]:
ls orvara/tests/data/gaia/

HIP003850.csv  HIP085653.csv  HIP095319.csv


In [12]:
ls orvara/tests/data/hip1/

003850.txt  085653.txt  95319.txt


In [13]:
ls orvara/tests/data/hip2/

HIP003850.d  HIP085653.d  HIP095319.d


### Now, let's run fit_orbit and plot_orbit commands in the root directory

We are going to run an example fit to HD 159062. Verifty that the RV data, relative astrometry data and absolute astrometry data are loaded in correctly to ensure a suceesful fit. Progress bar shows the progress of MCMC. 

In [3]:
!fit_orbit --output-dir orvara/tests/chains orvara/tests/config_HD159062.ini

Loading RV data from file orvara/tests/data/HD159062_RV.dat
Loaded data from 1 RV instruments.
Loading astrometric data for 1 planets
Loaded 3 relative astrometric data points from file orvara/tests/data/HD159062_relAST.txt
Loading absolute astrometry data for Hip 85653
Not using companion proper motion from Gaia.
Using ptemcee.
Running MCMC ... 
[###############################]100% 1%  2%  3%  4%  5%  6%  7%  8%  9% 10% 11% 12% 13% 14% 15% 16% 17% 18% 19% 20% 21% 22% 23% 24% 25% 26% 27% 28% 29% 30% 31% 32% 33% 34% 35% 36% 37% 38% 39% 40% 41% 42% 43% 44% 45% 46% 47% 48% 49% 50% 51% 52% 53% 54% 55% 56% 57% 58% 59% 60% 61% 62% 63% 64% 65% 66% 67% 68% 69% 70% 71% 72% 73% 74% 75% 76% 77% 78% 79% 80% 81% 82% 83% 84% 85% 86% 87% 88% 89% 90% 91% 92% 93% 94% 95% 96% 97% 98% 99%
Total Time: 409 seconds
Mean acceptance fraction (cold chain): 0.116571
Writing output to orvara/tests/chains/HD159062B_chain000.fits


Once you have the MCMC chain output by fit_orbit, feed it to McmcDataFile= in the config file. You can set a burn-in phase for the chain. After that, you are ready to go on to plot the orbits. plot_orbit will generate a suite of 7 plots relavant to RV and Astrometry, and a plot of the MCMC chain to breifly check convergence, and two files containting the beststep and posterior parameters. 

In [6]:
!plot_orbit --output-dir orvara/tests/plots/ orvara/tests/config_HD159062.ini

Generating plots for target HD159062B
Generating diagonstic plots to check convergence
Plotting Astrometry orbits, your plot is generated at orvara/tests/plots/
Plotting RV orbits, your plot is generated at orvara/tests/plots/
Plotting RV, your plot is generated at orvara/tests/plots/
Plotting Separation, your plot is generated at orvara/tests/plots/
Plotting Position Angle, your plot is generated at orvara/tests/plots/
Plotting Proper Motions, your plot is generated at orvara/tests/plots/
Plotting Corner plot, your plot is generated at orvara/tests/plots/
Saving beststep parameters to orvara/tests/plots/
Saving posterior parameters to orvara/tests/plots/


### Great! Your plots are generated in the default or specified output directory.

You can see the whole sets of plots in the output directory.

In [3]:
ls orvara/tests/plots/

Corner_HD159062B.pdf                  astrometric_orbit_HD159062B.pdf
Diagnostic_HD159062B.png              astrometric_prediction_HD159062B.pdf
PA_OC_HD159062B.pdf                   beststep_params_HD159062B.txt
Proper_Motions_HD159062B.pdf          posterior_params_HD159062B.txt
RV_OC_HD159062B_InstAll.pdf           relsep_OC_HD159062B.pdf
RV_orbit_HD159062B.pdf


We can load the parameter files from the fit:

In [ ]:
# %load orvara/tests/plots/beststep_params_HD159062B.txt
plx_ML  0.04618727907
pmra_ML  0.1695171527
pmdec_ML  0.08816530131
chisq_sep  0.7060916553
chisq_PA  2.141646391
chisq_H  4.077306854
chisq_HG  0.1247828772
chisq_G  0.2462001314
RV_ZP_0_ML  -1479.148349


In [ ]:
# %load orvara/tests/plots/posterior_params_HD159062B.txt
Jit (m/s)         ${1.26}_{-0.29}^{+0.33}$
Mpri (solar)         ${0.802}_{-0.050}^{+0.048}$
Msec (solar)         ${0.6057}_{-0.0069}^{+0.0077}$
a (AU)         ${56.8}_{-6.4}^{+4.4}$
sqesinw         ${-0.207}_{-0.087}^{+0.13}$
sqecosw         ${0.26}_{-0.19}^{+0.20}$
inclination (deg)         ${61.6}_{-2.9}^{+1.7}$
ascending node (deg)         ${134.5}_{-1.2}^{+2.2}$
mean longitude (deg)         ${147.3}_{-7.9}^{+5.7}$
parallax (mas)         ${46.1855}_{-0.0042}^{+0.0042}$
period (yrs)         ${361}_{-60}^{+44}$
argument of periastron (deg)         ${322}_{-47}^{+17}$
eccentricity         ${0.123}_{-0.077}^{+0.14}$
semimajor axis (mas)         ${2625}_{-296}^{+203}$
T0 (JD)         ${2519211}_{-43442}^{+44845}$
mass ratio         ${0.755}_{-0.044}^{+0.051}$
